# 1. Train data 전처리 

## 1) Importing libraries & dataset

드라이브 마운트만 숨겨진 셀 클릭해서 들어가셔야 하고 <br>
나머지는 한 번에 돌려주세요.

#### (1) importing libraries - 한 번에 통째로 클릭

In [ ]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

Cloning into 'khaiii'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 957 (delta 25), reused 49 (delta 19), pack-reused 877
Receiving objects: 100% (957/957), 33.03 MiB | 12.44 MiB/s, done.
Resolving deltas: 100% (383/383), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-

In [ ]:
from khaiii import KhaiiiApi
import pandas as pd
import numpy as np
import re

#### (2) 드라이브 마운트 - 공유드라이브에서 데이터셋 불러옵니다. 

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
X_train=pd.read_csv("/content/drive/Shareddrives/20-2 KUBIG 자연어처리 프로젝트/최종 트레이닝데이터/x_train.csv")
y_train=pd.read_csv("/content/drive/Shareddrives/20-2 KUBIG 자연어처리 프로젝트/최종 트레이닝데이터/y_train.csv")

#### (3) 데이터 확인하고, 나누어서 저장합니다

In [5]:
y_train = y_train.drop(['Unnamed: 0'],axis=1)
y_train.tail()

,label
99433,1
99434,1
99435,0
99436,0
99437,0


In [6]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99438 entries, 0 to 99437
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   label   99438 non-null  int64
dtypes: int64(1)
memory usage: 777.0 KB


In [7]:
X_train.head()

,Unnamed: 0,document
0,0,평점에 스포 달린 영화는 무조건 점
1,1,재밌습니다 잘 만든 영화라는
2,2,딱 점
3,3,만남과 헤어짐 그리고 재회
4,4,롱기스트 야드를 먼저 봐서 그런지 진짜 쓰레기 특히 주인공 진짜 맘에 안 든다


In [8]:
X_train = X_train.drop(['Unnamed: 0'],axis=1)
X_train

,document
0,평점에 스포 달린 영화는 무조건 점
1,재밌습니다 잘 만든 영화라는
2,딱 점
3,만남과 헤어짐 그리고 재회
4,롱기스트 야드를 먼저 봐서 그런지 진짜 쓰레기 특히 주인공 진짜 맘에 안 든다
...,...
99433,최고 다 년 전 영화라고 안 믿길 정도
99434,시간 아깝다고 할 정도의 실패작은 아닌 것 같다 박 진영의 연기도 생각 외로 괜찮았...
99435,오다기리 조는 맨날 나와
99436,붐 얼굴 보면 화가 나


In [9]:
X_train=X_train.dropna(how = 'any')
print(X_train.isnull().any())
print(X_train.info())

document    False
dtype: bool
<class 'pandas.core.frame.DataFrame'>
Int64Index: 99438 entries, 0 to 99437
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   document  99438 non-null  object
dtypes: object(1)
memory usage: 1.5+ MB
None


In [10]:
dat1=X_train[:30001]
dat2=X_train[30001:60001]
dat3=X_train[60001:90001]
dat4=X_train[90001:]

#### (4) 형태소 분석기 돌리기 

In [11]:
api = KhaiiiApi()

In [12]:
significant_tags = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'VX', 'MAG', 'MAJ', 'XSV', 'XSA']

In [13]:
def pos_text(texts):
    corpus = []
    for sent in texts:
        pos_tagged = ''
        for word in api.analyze(sent):
            for morph in word.morphs:
                if morph.tag in significant_tags:
                    pos_tagged += morph.lex + '/' + morph.tag + ' '
        corpus.append(pos_tagged.strip())
    return corpus

In [14]:
tagged_corpus1=pos_text(dat1["document"])

In [15]:
tagged_corpus2=pos_text(dat2["document"])

In [16]:
tagged_corpus3=pos_text(dat3["document"])

In [17]:
tagged_corpus4=pos_text(dat4["document"])

In [18]:
p1 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XS.')
p2 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XSA [가-힣A-Za-z0-9]+/VX')
p3 = re.compile('[가-힣A-Za-z0-9]+/VV')
p4 = re.compile('[가-힣A-Za-z0-9]+/VX')

In [19]:
tag_corp=tagged_corpus1+tagged_corpus2+tagged_corpus3+tagged_corpus4

## 2) preprocessing - 정규화 등

In [20]:
def stemming_text(text):
    corpus = []
    for sent in text:
        ori_sent = sent
        mached_terms = re.findall(p1, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        
        mached_terms = re.findall(p2, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                if tag != 'VX':
                    modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p3, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p4, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        corpus.append(ori_sent)
    return corpus

In [21]:
stemming_corpus = stemming_text(tag_corp)

In [22]:
for i in range(0, 30):
    print(stemming_corpus[i])

평점/NNG 스포/NNG 달리다/VV 영화/NNG 무조건/MAG 점/NNG
재미있/VA 잘/MAG 만들다/VV 영화/NNG
딱/MAG 점/NNG
만남/NNG 헤어지다/VV 그리고/MAJ 재회/NNG
롱/NNG 기스트/NNP 야/NNG 드/NNP 먼저/MAG 보다/VV 그렇/VA 진짜/NNG 쓰레기/NNG 특히/MAG 주인공/NNG 진짜/NNG 맘/NNG 안/MAG 들다/VV
오늘/NNG 고지전/NNG 다시/MAG 보다/VV 나라/NNG 위하다/VV 휴전/NNG 직전/NNG 싸우다/VV 전사하다/VV 참전/NNG 용사/NNG 감사/NNG 드다/VV 리/XSV
술/NNG 담배/NNG 더/MAG 해롭/VA 것/NNB 막장/NNG 드라마/NNG
정말/MAG 좋/VA 특히/MAG 영상/NNG 음악/NNG
ㅡ/NNP
예고편/NNG 낚이다/VV 최악/NNG 영화/NNG 중/NNB
현실/NNG 재밌/VA
하/XSA 영화/NNG 만들다/VV 내다/VV 수/NNB 없/VA 구로/NNP 사와/NNG 재능/NNG
우/MAG 굳ㅋ/MAG 재미/NNG 있/VA
쩌르다/VV 진짜/MAG 국영/NNG 형/NNG 최고/NNG
쓰레기/NNG 영화/NNG 다/MAG 있다/VV 인물/NNG 관계/NNG 정리/NNG 안/MAG 하다/VV 주다/VV 줄거리/NNG 없/VA 내용/NNG ㅉ/NNG
무섭/VA 오페라/NNG 유령/NNG 프레데/NNP 터/NNG
결혼하다/VV 보다/VV 미혼/NNG 때/NNG 또/MAG 감동/NNG 받다/VV 영화/NNG
캐서린/NNP 제타/NNG 존스/NNP 보톡스/NNG 자연스럽다/VV 영화/NNG 마치/MAG 전성기/NNG 훨씬/MAG 지나다/VV 톱스타/NNG 뒤늦/VA 공연/NNG 하다/VV 듯하다/VV 느낌/NNG 마디/NNG 철/NNG 지나다/VV 급/NNG 액션/NNG
이상하다/VV 것/NNB 만들다/VV 것/NNB 왠지/MAG 시간/NNG 지나다/VV 허접하다/VV 지다/VV
드라마/NNG 재밌/VA 보다/VV 것/NNB 없/VA 것/NNB 같/VA 구다

In [23]:
def text_cleaning(text):
   hangul=re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]+")
   result=hangul.sub('', text)
   return result

In [24]:
for sent in stemming_corpus:
  map(text_cleaning, sent)

In [25]:
corpus=[]
for i in range(0, len(stemming_corpus)):
  corpus.append(text_cleaning(stemming_corpus[i]))

In [26]:
clean_corpus=corpus

In [27]:
corpus=[]
for sent in clean_corpus:
  corpus.append(sent.split())

In [28]:
corpus_comma=corpus

In [29]:
for i in range(0, 30):
    print(corpus_comma[i])

['평점', '스포', '달리다', '영화', '무조건', '점']
['재미있', '잘', '만들다', '영화']
['딱', '점']
['만남', '헤어지다', '그리고', '재회']
['롱', '기스트', '야', '드', '먼저', '보다', '그렇', '진짜', '쓰레기', '특히', '주인공', '진짜', '맘', '안', '들다']
['오늘', '고지전', '다시', '보다', '나라', '위하다', '휴전', '직전', '싸우다', '전사하다', '참전', '용사', '감사', '드다', '리']
['술', '담배', '더', '해롭', '것', '막장', '드라마']
['정말', '좋', '특히', '영상', '음악']
['ㅡ']
['예고편', '낚이다', '최악', '영화', '중']
['현실', '재밌']
['하', '영화', '만들다', '내다', '수', '없', '구로', '사와', '재능']
['우', '굳ㅋ', '재미', '있']
['쩌르다', '진짜', '국영', '형', '최고']
['쓰레기', '영화', '다', '있다', '인물', '관계', '정리', '안', '하다', '주다', '줄거리', '없', '내용', 'ㅉ']
['무섭', '오페라', '유령', '프레데', '터']
['결혼하다', '보다', '미혼', '때', '또', '감동', '받다', '영화']
['캐서린', '제타', '존스', '보톡스', '자연스럽다', '영화', '마치', '전성기', '훨씬', '지나다', '톱스타', '뒤늦', '공연', '하다', '듯하다', '느낌', '마디', '철', '지나다', '급', '액션']
['이상하다', '것', '만들다', '것', '왠지', '시간', '지나다', '허접하다', '지다']
['드라마', '재밌', '보다', '것', '없', '것', '같', '구다']
['결말', '좀', '슬프', '다', '전두엽', '수순', '끝내', '받다', '맥머피', '이제', '예전', '맥머피', '어쩌다',

In [30]:
path="/content/drive/Shareddrives/20-2 KUBIG 자연어처리 프로젝트/korean_stopwords.txt"
with open(path, encoding='utf-8') as f:
  stopwords=f.readlines()
stopwords=[x.strip() for x in stopwords]

In [31]:
corpus=[]
for sentence in corpus_comma:
    temp_X = [word for word in sentence if not word in stopwords]
    corpus.append(temp_X)

In [32]:
removed_stopword_corpus = corpus

In [33]:
for i in range(0, 30):
    print(removed_stopword_corpus[i])

['평점', '스포', '달리다', '영화', '무조건', '점']
['재미있', '잘', '만들다', '영화']
['점']
['만남', '헤어지다', '재회']
['롱', '기스트', '드', '먼저', '보다', '그렇', '진짜', '쓰레기', '특히', '주인공', '진짜', '맘', '안', '들다']
['오늘', '고지전', '다시', '보다', '나라', '위하다', '휴전', '직전', '싸우다', '전사하다', '참전', '용사', '감사', '드다', '리']
['술', '담배', '더', '해롭', '막장', '드라마']
['정말', '좋', '특히', '영상', '음악']
['ㅡ']
['예고편', '낚이다', '최악', '영화', '중']
['현실', '재밌']
['영화', '만들다', '내다', '수', '없', '구로', '사와', '재능']
['우', '굳ㅋ', '재미', '있']
['쩌르다', '진짜', '국영', '형', '최고']
['쓰레기', '영화', '다', '인물', '관계', '정리', '안', '하다', '주다', '줄거리', '없', '내용', 'ㅉ']
['무섭', '오페라', '유령', '프레데', '터']
['결혼하다', '보다', '미혼', '감동', '받다', '영화']
['캐서린', '제타', '존스', '보톡스', '자연스럽다', '영화', '전성기', '지나다', '톱스타', '뒤늦', '공연', '하다', '듯하다', '느낌', '마디', '철', '지나다', '급', '액션']
['이상하다', '만들다', '왠지', '지나다', '허접하다', '지다']
['드라마', '재밌', '보다', '없', '같', '구다']
['결말', '슬프', '다', '전두엽', '수순', '끝내', '받다', '맥머피', '이제', '예전', '맥머피', '어쩌다', '수', '없이', '치프', '죽이다', '주다', '영혼', '탈출', '시도하다', '하다', '거', '같', '빌리', '맥머피', '죽음', 

## 3) 워드 임베딩

In [34]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=removed_stopword_corpus, size=100, window=5, min_count=5, workers=4, sg=1)

In [35]:
model.wv.vectors.shape

(10620, 100)

In [36]:
from gensim.models import KeyedVectors
model.wv.save_word2vec_format('khaii_skipgram') #모델 저장

# 2. Test data 전처리

### 2) 공유 드라이브 마운트

In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
test= pd.read_csv("/content/drive/Shareddrives/20-2 KUBIG 자연어처리 프로젝트/Test_Real_final.csv")
test

,label,0
0,1,굳 ㅋ
1,0,NaN
2,0,뭐 야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아
3,0,지루하지는 않은데 완전 막장 임 돈 주고 보기에는
4,0,만 아니었어도 별 다섯 개 줬을 텐데 왜 로 나와서 제 심기를 불편하게 하죠
...,...,...
52632,1,욕심과 오해가 불러낸 비극과 절망
52633,1,나쁜 벌시키 들 어린 시절 가장 기억에 남는 영화
52634,0,후지다는 말 이럴 때 써야지
52635,1,시리즈 중 최고라고 생각 번도 넘게 봤지만 다른 시리즈에서 느낄 수 없는 스펙타클함...


In [39]:
test = test.dropna(how = 'any')

In [40]:
print(test.isnull().values.any())

False


### 3) 형태소 분석기 Khaii

In [41]:
api = KhaiiiApi()

In [42]:
significant_tags = ['NNG', 'NNP', 'NNB', 'VV', 'VA', 'VX', 'MAG', 'MAJ', 'XSV', 'XSA']

In [43]:
def pos_text(texts):
    corpus = []
    for sent in texts:
        pos_tagged = ''
        for word in api.analyze(sent):
            for morph in word.morphs:
                if morph.tag in significant_tags:
                    pos_tagged += morph.lex + '/' + morph.tag + ' '
        corpus.append(pos_tagged.strip())
    return corpus

In [44]:
tagged_corpus=pos_text(test["0"])

In [45]:
len(tagged_corpus)

52244

### 4) 정규화

In [46]:
p1 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XS.')
p2 = re.compile('[가-힣A-Za-z0-9]+/NN. [가-힣A-Za-z0-9]+/XSA [가-힣A-Za-z0-9]+/VX')
p3 = re.compile('[가-힣A-Za-z0-9]+/VV')
p4 = re.compile('[가-힣A-Za-z0-9]+/VX')

In [47]:
def stemming_text(text):
    corpus = []
    for sent in text:
        ori_sent = sent
        mached_terms = re.findall(p1, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        
        mached_terms = re.findall(p2, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                if tag != 'VX':
                    modi_terms += lemma
            modi_terms += '다/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p3, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)

        mached_terms = re.findall(p4, ori_sent)
        for terms in mached_terms:
            ori_terms = terms
            modi_terms = ''
            for term in terms.split(' '):
                lemma = term.split('/')[0]
                tag = term.split('/')[-1]
                modi_terms += lemma
            if '다' != modi_terms[-1]:
                modi_terms += '다'
            modi_terms += '/VV'
            ori_sent = ori_sent.replace(ori_terms, modi_terms)
        corpus.append(ori_sent)
    return corpus

In [48]:
stemming_corpus = stemming_text(tagged_corpus)

In [49]:
for i in range(0, 30):
    print(stemming_corpus[i])

굳/NNG ㅋ/NNG
평점/NNG 나쁘/VA 않다/VV 점/NNB 짜다/VV 더더욱/MAG 않다/VV
하/XSA 않다/VV 완전/NNG 막장/NNG 임/NNG 돈/NNG 주다/VV 보다/VV
만/NNG 개/NNB 주다/VV 터/NNB 왜/MAG 로/NNP 나오다/VV 심기/NNG 불편하다/VV 하다/VV
음악/NNG 주가/NNG 되다/VV 최고/NNG 음악/NNG 영화/NNG
하/XSA 쓰레기/NNG
마치/MAG 미국/NNP 애니/NNG 튀어나오다/VV 듯하다/VV 창의력/NNG 없/VA 로봇/NNG 디자인/NNG 가다/VV 고개/NNG 젖다/VV 하다/VV
갈수록/MAG 개판되다/VV 중국/NNP 영화하다/VV 내용/NNG 없/VA 폼/NNG 잡다/VV 끝나다/VV 말/NNG 안/MAG 되다/VV 무기하다/VV 남무/NNG 그립/VA 동사/NNG 서독/NNP 같/VA 영화/NNG 류아류작/NNG
이별/NNG 아픔/NNG 뒤/NNG 찾아오다/VV 새롭/VA 인연/NNG 기/NNG 쁨/NNG 사람/NNG 그렇/VA 않다/VV
괜찮/VA 오랜/NNG 만/MAJ 포켓몬스터/NNG 잼밌/VA
한국독/NNP 립영화/NNG 한계/NNG 그렇/VA 아버지/NNG 되다/VV 비교되다/VV
청춘/NNG 아름답/VA 아름답/VA 이성/NNG 흔들다/VV 놓다/VV 다/MAG 찰나/NNG 아름답/VA 잘/MAG 포착하다/VV 하/XSA 아름답/VA 수채화/NNG 같/VA 퀴다/VV 어/NNG 영화/NNG
눈/NNG 보이다/VV 반전/NNG 영화/NNG 흡인력/NNG 사라다/VV 지/MAG 않다/VV
스토리/NNG 연출/NNG 연기/NNG 비주얼/NNG 등/NNB 영화/NNG 기/NNG 보다/VV 차/NNG 안/MAG 되/XSV 영화/NNG 평/NNG 하다/VV 영화/NNG 찍다/VV 김문옥/NNP 감독/NNG 영화/NNG 경력/NNG 조무래기/NNG 영화/NNG 평론하다/VV 같/VA 마인드/NNG 빠지다/VV 있다/VV
소위/MAG ㅈ/NNG 문가/NNG 평점/NNG
최고/NN

In [50]:
len(stemming_corpus)

52244

In [51]:
def text_cleaning(text):
   hangul=re.compile("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]+")
   result=hangul.sub('', text)
   return result

In [52]:
for sent in stemming_corpus:
  map(text_cleaning, sent)

In [53]:
corpus=[]
for i in range(0, len(stemming_corpus)):
  corpus.append(text_cleaning(stemming_corpus[i]))

In [54]:
clean_corpus = corpus

In [55]:
corpus=[]
for sent in clean_corpus:
  corpus.append(sent.split())

In [56]:
corpus_comma=corpus

In [57]:
for i in range(0, 30):
    print(corpus_comma[i])

['굳', 'ㅋ']
['평점', '나쁘', '않다', '점', '짜다', '더더욱', '않다']
['하', '않다', '완전', '막장', '임', '돈', '주다', '보다']
['만', '개', '주다', '터', '왜', '로', '나오다', '심기', '불편하다', '하다']
['음악', '주가', '되다', '최고', '음악', '영화']
['하', '쓰레기']
['마치', '미국', '애니', '튀어나오다', '듯하다', '창의력', '없', '로봇', '디자인', '가다', '고개', '젖다', '하다']
['갈수록', '개판되다', '중국', '영화하다', '내용', '없', '폼', '잡다', '끝나다', '말', '안', '되다', '무기하다', '남무', '그립', '동사', '서독', '같', '영화', '류아류작']
['이별', '아픔', '뒤', '찾아오다', '새롭', '인연', '기', '쁨', '사람', '그렇', '않다']
['괜찮', '오랜', '만', '포켓몬스터', '잼밌']
['한국독', '립영화', '한계', '그렇', '아버지', '되다', '비교되다']
['청춘', '아름답', '아름답', '이성', '흔들다', '놓다', '다', '찰나', '아름답', '잘', '포착하다', '하', '아름답', '수채화', '같', '퀴다', '어', '영화']
['눈', '보이다', '반전', '영화', '흡인력', '사라다', '지', '않다']
['스토리', '연출', '연기', '비주얼', '등', '영화', '기', '보다', '차', '안', '되', '영화', '평', '하다', '영화', '찍다', '김문옥', '감독', '영화', '경력', '조무래기', '영화', '평론하다', '같', '마인드', '빠지다', '있다']
['소위', 'ㅈ', '문가', '평점']
['최고']
['발연기', '도저히', '못', '보다', '진', '짜', '이렇', '연기', '못', '하', '거', '상상', '못', '하

In [58]:
path="/content/drive/Shareddrives/20-2 KUBIG 자연어처리 프로젝트/korean_stopwords.txt"
with open(path, encoding='utf-8') as f:
  stopwords=f.readlines()
stopwords=[x.strip() for x in stopwords]

In [59]:
corpus=[]
for sentence in corpus_comma:
    temp_X = [word for word in sentence if not word in stopwords]
    corpus.append(temp_X)

In [60]:
removed_stopword_corpus2 = corpus

In [61]:
for i in range(0, 30):
    print(removed_stopword_corpus2[i])

['굳', 'ㅋ']
['평점', '나쁘', '않다', '점', '짜다', '더더욱', '않다']
['않다', '완전', '막장', '임', '돈', '주다', '보다']
['만', '개', '주다', '터', '나오다', '심기', '불편하다', '하다']
['음악', '주가', '되다', '최고', '음악', '영화']
['쓰레기']
['미국', '애니', '튀어나오다', '듯하다', '창의력', '없', '로봇', '디자인', '가다', '고개', '젖다', '하다']
['갈수록', '개판되다', '중국', '영화하다', '내용', '없', '폼', '잡다', '끝나다', '말', '안', '되다', '무기하다', '남무', '그립', '동사', '서독', '같', '영화', '류아류작']
['이별', '아픔', '뒤', '찾아오다', '새롭', '인연', '기', '쁨', '사람', '그렇', '않다']
['괜찮', '오랜', '만', '포켓몬스터', '잼밌']
['한국독', '립영화', '한계', '그렇', '아버지', '되다', '비교되다']
['청춘', '아름답', '아름답', '이성', '흔들다', '놓다', '다', '찰나', '아름답', '잘', '포착하다', '아름답', '수채화', '같', '퀴다', '영화']
['눈', '보이다', '반전', '영화', '흡인력', '사라다', '지', '않다']
['스토리', '연출', '연기', '비주얼', '영화', '기', '보다', '차', '안', '되', '영화', '평', '하다', '영화', '찍다', '김문옥', '감독', '영화', '경력', '조무래기', '영화', '평론하다', '같', '마인드', '빠지다']
['소위', 'ㅈ', '문가', '평점']
['최고']
['발연기', '도저히', '못', '보다', '진', '짜', '이렇', '연기', '못', '거', '상상', '못']
['이스']
['재미', '없', '거', '우리다', '먹다', '챔프', '방송', '번', 

### 5) 워드임베딩 - Skip-gram 

In [62]:
Test = removed_stopword_corpus2

In [63]:
for line in Test[:3]: # 샘플 3개만 출력
    print(line)

['굳', 'ㅋ']
['평점', '나쁘', '않다', '점', '짜다', '더더욱', '않다']
['않다', '완전', '막장', '임', '돈', '주다', '보다']


In [64]:
from gensim.models.word2vec import Word2Vec
model2 = Word2Vec(sentences=Test, size=100, window=5, min_count=5, workers=4, sg=1)

In [65]:
model2.wv.vectors.shape

(7206, 100)

In [66]:
from gensim.models.keyedvectors import KeyedVectors
model2.wv.save_word2vec_format('my.embedding', binary=False) #모델 저장
loaded_model = KeyedVectors.load_word2vec_format('my.embedding', binary=False, encoding='utf-8')

In [67]:
result = loaded_model.most_similar('별로')
print(result)

[('그닥', 0.8815822601318359), ('전편', 0.813557505607605), ('머다', 0.805023193359375), ('어이없', 0.8049728870391846), ('뻔하', 0.7962434887886047), ('탄', 0.7924604415893555), ('감흥', 0.79204261302948), ('유치하다', 0.7898272275924683), ('줄거리', 0.7888825535774231), ('솔직히', 0.7856125831604004)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# 3. LSTM

In [68]:
import numpy
import tensorflow as tf
import matplotlib.pyplot as plt

In [69]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [70]:

t = Tokenizer()
t.fit_on_texts(removed_stopword_corpus)
vocab_size = len(t.word_index) + 1
print(vocab_size)



44701


In [71]:
X_encoded = t.texts_to_sequences(removed_stopword_corpus)
print(X_encoded[:10])

[[25, 2239, 776, 1, 651, 17], [20, 21, 16, 1], [17], [1895, 1653, 5514], [2780, 21548, 288, 749, 2, 84, 24, 49, 225, 98, 24, 346, 10, 61], [317, 15543, 54, 2, 460, 145, 12520, 3192, 614, 12521, 9293, 9294, 1654, 1218, 123], [1534, 1655, 28, 21549, 212, 33], [8, 6, 225, 206, 144], [121], [743, 495, 86, 1, 39]]


In [72]:
max_len=max(len(l) for l in X_encoded)
print(max_len)


66


In [73]:
X_train=pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train=np.array(y_train)
print(X_train)
embedding_matrix = np.zeros((vocab_size, 100))

def get_vector(word):
    if word in model:
        return model[word]
    else:
        return None

for word, i in t.word_index.items(): 
    temp = get_vector(word) 
    if temp is not None: 
        embedding_matrix[i] = temp

[[   25  2239   776 ...     0     0     0]
 [   20    21    16 ...     0     0     0]
 [   17     0     0 ...     0     0     0]
 ...
 [10343  3498  1864 ...     0     0     0]
 [ 5904   369     2 ...     0     0     0]
 [  526   220     2 ...     0     0     0]]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [74]:
X_train

array([[   25,  2239,   776, ...,     0,     0,     0],
       [   20,    21,    16, ...,     0,     0,     0],
       [   17,     0,     0, ...,     0,     0,     0],
       ...,
       [10343,  3498,  1864, ...,     0,     0,     0],
       [ 5904,   369,     2, ...,     0,     0,     0],
       [  526,   220,     2, ...,     0,     0,     0]], dtype=int32)

In [75]:

t2 = Tokenizer()
t2.fit_on_texts(removed_stopword_corpus2)
vocab_size2 = len(t2.word_index) + 1
print(vocab_size2)



37744


In [76]:
X_encoded2 = t2.texts_to_sequences(removed_stopword_corpus2)
print(X_encoded2[:10])

[[907, 23], [24, 273, 15, 20, 548, 5498, 15], [15, 79, 254, 188, 76, 8, 2], [122, 89, 8, 243, 21, 15848, 597, 3], [152, 15849, 9, 27, 152, 1], [57], [270, 302, 2505, 186, 15850, 4, 1527, 2741, 16, 3365, 2300, 3], [1042, 5499, 454, 281, 44, 4, 2397, 300, 141, 40, 10, 9, 15851, 15852, 808, 10934, 15853, 13, 1, 15854], [2869, 1263, 294, 2506, 419, 2398, 64, 2021, 26, 71, 15], [101, 4932, 122, 10935, 15855]]


In [77]:
max_len2=max(len(l) for l in X_encoded2)
print(max_len2)


448


In [78]:
y_test=test["label"]

X_test=pad_sequences(X_encoded2, maxlen=max_len2, padding='post')
y_test=np.array(y_test)
print(X_test)

'''
embedding_matrix2 = np.zeros((vocab_size2, 100))


def get_vector(word2):
    if word2 in model2:
        return model2[word2]
    else:
        return None

for word2, i in t2.word_index.items(): 
    temp2 = get_vector(word2) 
    if temp2 is not None: 
        embedding_matrix2[i] = temp2
'''

[[ 907   23    0 ...    0    0    0]
 [  24  273   15 ...    0    0    0]
 [  15   79  254 ...    0    0    0]
 ...
 [4261   40   59 ...    0    0    0]
 [ 192   33   27 ...    0    0    0]
 [  48   54    2 ...    0    0    0]]


'\nembedding_matrix2 = np.zeros((vocab_size2, 100))\n\n\ndef get_vector(word2):\n    if word2 in model2:\n        return model2[word2]\n    else:\n        return None\n\nfor word2, i in t2.word_index.items(): \n    temp2 = get_vector(word2) \n    if temp2 is not None: \n        embedding_matrix2[i] = temp2\n'

In [79]:
from keras.layers import Embedding, Dense, LSTM, Flatten
from keras.models import Sequential 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

In [80]:
model3 = Sequential() 
model3.add(Embedding(vocab_size, 100, weights=[embedding_matrix],input_length=max_len, trainable=False)) 
model3.add(LSTM(256)) 
model3.add(Dense(1, activation='sigmoid'))

In [81]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [82]:
model3.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model3.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=256, validation_split=0.2)

#여기서 es 작동이 안 되는데 왜 그런지 모르겠어요ㅠ

Epoch 1/15
311/311 [==============================] - ETA: 0s - loss: 0.6576 - acc: 0.5928
Epoch 00001: val_acc improved from -inf to 0.63310, saving model to best_model.h5
311/311 [==============================] - 321s 1s/step - loss: 0.6576 - acc: 0.5928 - val_loss: 0.6201 - val_acc: 0.6331
Epoch 2/15
311/311 [==============================] - ETA: 0s - loss: 0.5171 - acc: 0.7573
Epoch 00002: val_acc improved from 0.63310 to 0.79138, saving model to best_model.h5
311/311 [==============================] - 316s 1s/step - loss: 0.5171 - acc: 0.7573 - val_loss: 0.4667 - val_acc: 0.7914
Epoch 3/15
311/311 [==============================] - ETA: 0s - loss: 0.4534 - acc: 0.7935
Epoch 00003: val_acc improved from 0.79138 to 0.80164, saving model to best_model.h5
311/311 [==============================] - 306s 983ms/step - loss: 0.4534 - acc: 0.7935 - val_loss: 0.4369 - val_acc: 0.8016
Epoch 4/15
311/311 [==============================] - ETA: 0s - loss: 0.4347 - acc: 0.8012
Epoch 00004: va

In [83]:
loaded_model = load_model('best_model.h5')
print("\n 테스트 정확도: {:.2f}%".format(loaded_model.evaluate(X_test,y_test)[1]*100))

1633/1633 [==============================] - 767s 469ms/step - loss: 0.7932 - acc: 0.5473

 테스트 정확도: 54.73%
